In [1]:
# Cell 1: 导入依赖
import os
import shutil
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

print("依赖导入完成!")

依赖导入完成!


In [ ]:
# Cell 2: 设置数据路径
# 修改这些路径为你的实际路径
source_base_dir = r"C:\path\to\your\cifar100"  # 原始数据集根目录
target_base_dir = r"C:\path\to\your\cifar100_organized"  # 重组后的目录

# 训练集和测试集路径
train_source = os.path.join(source_base_dir, "train")
test_source = os.path.join(source_base_dir, "test")

train_target = os.path.join(target_base_dir, "train")
test_target = os.path.join(target_base_dir, "test")

print(f"源训练集路径: {train_source}")
print(f"源测试集路径: {test_source}")
print(f"目标训练集路径: {train_target}")
print(f"目标测试集路径: {test_target}")

In [3]:
import os
import pickle
import numpy as np
from PIL import Image
from tqdm import tqdm  # 进度条（可选）

# -------------------------- 配置路径 --------------------------
# 原始CIFAR-100数据集路径（根据你的实际情况修改）
BASE_DIR = r"E:\cifar100\CIFAR-10-100(含png图)\cifar-100-python"
# 输出目录（整理后的类别文件夹存放处）
OUTPUT_BASE = r"E:\cifar100\CIFAR-10-100(含png图)\cifar-100-remake"


# -------------------------- 读取类别名称 --------------------------
def load_fine_labels(meta_path):
    """从meta文件中读取细粒度类别名称（100类）"""
    with open(meta_path, "rb") as f:
        meta = pickle.load(f, encoding="latin1")  # CIFAR-100的pkl用latin1编码
    return meta["fine_label_names"]  # 返回100个类别名称，如["apple", "aquarium_fish", ...]


# -------------------------- 处理单个pkl文件（训练/测试） --------------------------
def process_dataset(pkl_path, output_dir, label_names):
    """
    解析pkl文件中的图像和标签，保存到对应类别文件夹
    :param pkl_path: 训练/测试集pkl文件路径（如BASE_DIR/train）
    :param output_dir: 输出基础目录（如OUTPUT_BASE/train）
    :param label_names: 类别名称列表
    """
    # 加载pkl数据
    with open(pkl_path, "rb") as f:
        data = pickle.load(f, encoding="latin1")
    
    # 解析数据：images是(50000, 3072)的向量 → 转换为(50000, 32, 32, 3)的图像数组
    images = data["data"].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # 关键：还原RGB通道
    labels = data["fine_labels"]  # 每个样本的细粒度标签（0-99）
    
    # 遍历每个样本，保存到对应类别文件夹
    for idx, (img_array, label) in enumerate(tqdm(zip(images, labels), total=len(images))):
        # 获取类别名称和文件夹路径
        label_name = label_names[label]
        class_dir = os.path.join(output_dir, label_name)
        os.makedirs(class_dir, exist_ok=True)  # 若文件夹不存在则创建
        
        # 保存图像：文件名格式为“train_0.png”“test_1.png”（避免重复）
        img_filename = f"{os.path.basename(pkl_path)[:-4]}_{idx}.png"
        img_path = os.path.join(class_dir, img_filename)
        
        # 将numpy数组转换为PIL图像并保存
        img = Image.fromarray(img_array)
        img.save(img_path)
    
    print(f"✅ 处理完成：{os.path.basename(pkl_path)} → 保存到{output_dir}，共{len(images)}张图")


# -------------------------- 主流程 --------------------------
if __name__ == "__main__":
    # 1. 加载类别名称
    meta_path = os.path.join(BASE_DIR, "meta")
    fine_label_names = load_fine_labels(meta_path)
    print(f"共加载{len(fine_label_names)}个类别：{fine_label_names[:10]}...")  # 打印前10类预览
    
    # 2. 创建输出目录结构
    train_output = os.path.join(OUTPUT_BASE, "train")  # 训练集输出目录
    test_output = os.path.join(OUTPUT_BASE, "test")    # 测试集输出目录
    os.makedirs(train_output, exist_ok=True)
    os.makedirs(test_output, exist_ok=True)
    
    # 为每个类别创建子文件夹（训练/测试都创建）
    for label in fine_label_names:
        os.makedirs(os.path.join(train_output, label), exist_ok=True)
        os.makedirs(os.path.join(test_output, label), exist_ok=True)
    
    # 3. 处理训练集和测试集
    train_pkl = os.path.join(BASE_DIR, "train")
    test_pkl = os.path.join(BASE_DIR, "test")
    
    process_dataset(train_pkl, train_output, fine_label_names)  # 处理训练集
    process_dataset(test_pkl, test_output, fine_label_names)    # 处理测试集
    
    print(f"所有数据整理完成！结果存放在：{OUTPUT_BASE}")
    print(f"结构示例：{OUTPUT_BASE}/train/apple/ → 存放所有“苹果”类别的训练图")

共加载100个类别：['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle']...


100%|██████████| 50000/50000 [00:27<00:00, 1842.82it/s]


✅ 处理完成：train → 保存到E:\cifar100\CIFAR-10-100(含png图)\cifar-100-remake\train，共50000张图


100%|██████████| 10000/10000 [00:05<00:00, 1734.60it/s]

✅ 处理完成：test → 保存到E:\cifar100\CIFAR-10-100(含png图)\cifar-100-remake\test，共10000张图
所有数据整理完成！结果存放在：E:\cifar100\CIFAR-10-100(含png图)\cifar-100-remake
结构示例：E:\cifar100\CIFAR-10-100(含png图)\cifar-100-remake/train/apple/ → 存放所有“苹果”类别的训练图
